### find best correlated features for each route and apply dynamic modelling to find R Squared values

In [26]:
frame = {}
corr= {}
last_dict = {}
for r in top10_routes['route']:
    frame[r] = df_pivot[df_pivot['route']==r]
    feat_select_cols = df_pivot.columns.tolist()[4:]
    spearman_corrx = {}
    for i in feat_select_cols:
        spearman_corrx[i] = frame[r].loc[:,i]
        spearman_df = pd.DataFrame.from_dict(spearman_corrx)
        corr[r] = spearman_df.corr(method='spearman').abs()

for k,v in corr.items():
    last_dict[k]=(v['res_cnt'])
    
xx = pd.DataFrame(last_dict)

final_dict = defaultdict(list)
for i in xx.columns.tolist():
    for r in xx.index:
        if (xx.loc[r,i]>0.2) & (xx.loc[r,i]<1):
            final_dict[i].append(r)
            
            

sco = {}
for rou,val in final_dict.items():
    X = df_pivot[df_pivot['route']==rou][val]
    y = df_pivot[df_pivot['route']==rou]['res_cnt']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    xgb_r = xg.XGBRegressor(n_estimators = 100,max_depth=4, seed = 5)
    xgb_r.fit(X_train, y_train)

    pred = xgb_r.predict(X_test) 
    sco[rou] = (r2_score(y_test, pred))
    
sco

{'Ankara|İstanbul': -0.0026141410324140857,
 'İstanbul|Ankara': -0.04669100146654204,
 'İzmir|İstanbul': 0.28013295100434565,
 'Ankara|Adana': -0.018854415391881263,
 'Bursa|İstanbul': 0.3841777413372758,
 'İstanbul|İzmir': 0.5870793490915198,
 'Adana|Ankara': 0.6627196560955554,
 'Ankara|İzmir': -0.6383169408878042,
 'Ankara|Mersin': 0.13059810925350424,
 'Mersin|Ankara': 0.5277491888103039,
 'İstanbul|Kocaeli': 0.36372888581786766,
 'İstanbul|Bursa': 0.41012283417595186,
 'Ankara|Antalya': 0.5096715727731307}

---